In [1]:
import os
import openai
from langchain.chat_models import ChatOpenAI   
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain   
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
# from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType   
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI   
from langchain.indexes import VectorstoreIndexCreator
openai_api_key = "sk-None-XS76dEz4IURDhsv2AJb9T3BlbkFJYoWnTsDhATqUPkspJY6n"

## Model Defintion 

In [2]:
llm_model = "gpt-3.5-turbo"

In [3]:
llm = ChatOpenAI(openai_api_key=openai_api_key , temperature=0.9, model=llm_model) 



/home/mohammad/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


#### Tell me about a brand

In [ ]:

prompt1 = ChatPromptTemplate.from_template("Tell me more about the brand {brand_name}. What are their popular products, target audience, and unique features? if it is not a fashion brand state that you are a fashion expert ")
chain1 = LLMChain(llm=llm, prompt=prompt1) 

#### Tell me about a brand that offers a product

In [ ]:

prompt2 = ChatPromptTemplate.from_template(
    "I'm looking to explore some fashion and dress brands that offer high-quality {product}. Could you provide a list of well-known and emerging brands that excel in this category? Please include information about their style, target audience, and any unique features or values that set them apart "
)

chain_2 = LLMChain(llm=llm, prompt=prompt2)     


#### what to were on occassion 

In [ ]:
prompt3 = ChatPromptTemplate.from_template( 
    """As a fashion expert, please recommend an outfit for the following situation: {situation}. Include details about the type of clothing, colors, accessories, and any additional styling tips to ensure the outfit is stylish and appropriate for the occasion.""" ) 
  
chain_3 = LLMChain(llm=llm, prompt=prompt3)  


#### Agents

In [ ]:
tools = load_tools(["wikipedia"], llm=llm)   
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    max_tokens=150)   


#### Chain to access csv 

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import CSVLoader
from langchain.llms import OpenAI
from IPython.display import Markdown


# Path to your CSV file
input_file = "/home/mohammad/Downloads/stores.csv"

# Initialize the loader with the CSV file
loader = CSVLoader(file_path=input_file)

# Initialize the embedding model
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Create the vector store index with the embedding model
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding_model
).from_loaders([loader])

# Define the query
query = "Please list all your shirts with sun protection in a table in markdown and summarize each one."

# Initialize the language model
llm_replacement_model = OpenAI(temperature=0, model='gpt-3.5-turbo-instruct', openai_api_key=openai_api_key)

# Perform the query
response = index.query(query, llm=llm_replacement_model)

# Display the response
display(Markdown(response))


/home/mohammad/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-07-21 00:21:23.057106: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 00:21:23.097574: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-21 00:21:23.416663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use avail



| ProductID | ProductName | ProductBrand | Gender | Price (INR) | NumImages | Description | PrimaryColor | Store |
|-----------|-------------|--------------|--------|-------------|-----------|-------------|--------------|-------|
| 1000795 | Being Human Clothing Navy Printed Casual Slim Shirt | Being Human | Men | 1079 | 6 | Navy blue printed casual shirt with spread collar, full button placket, two pockets, long sleeves, and curved hem | Navy | Store One |
| 10000245 | Parx Men Green Printed Polo Collar T-shirt | Parx | Men | 629 | 5 | Green printed T-shirt with polo collar and short sleeves | Green | Store Three |
| 10017899 | Parx Men Yellow & Off-White Slim Fit Checked Casual Shirt | Parx | Men | 647 | 4 | Yellow and off-white checked casual shirt with spread collar, long sleeves, button placket, curved hem, and one patch pocket | Yellow | Store One |
| 10017831 | Parx Men Blue Slim Fit Printed Casual Shirt | Parx | Men | 779 | 5 | Blue printed casual shirt with spread collar, long sleeves, button placket, curved hem, and

In [ ]:
def route_request(request):
    if "brand" in request:
        return chain1.generate_response(request)
    elif "product" in request:
        return chain_2.generate_response(request)
    elif "situation" in request:
        return chain_3.generate_response(request)
    else:
        return agent.generate_response(request)  